In [ ]:
import tensorflow as tf 
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt



In [ ]:
with open('/content/intents.json') as content:
  data1=json.load(content)

In [ ]:
tags=[]
inputs=[]
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['patterns']:
    inputs.append(lines)
    tags.append(intent['tag'])

In [ ]:
tags

['greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'greeting',
 'goodbye',
 'goodbye',
 'goodbye',
 'thanks',
 'thanks',
 'thanks',
 'thanks',
 'items',
 'items',
 'items',
 'payments',
 'payments',
 'payments',
 'payments',
 'delivery',
 'delivery',
 'delivery',
 'funny',
 'funny',
 'funny']

In [ ]:
responses

{'delivery': ['Delivery takes 2-4 days', 'Shipping takes 2-4 days'],
 'funny': ['Why did the hipster burn his mouth? He drank the coffee before it was cool.',
  'What did the buffalo say when his son left for college? Bison.'],
 'goodbye': ['See you later, thanks for visiting',
  'Have a nice day',
  'Bye! Come back again soon.'],
 'greeting': ['Hey :-)',
  'Hello, thanks for visiting',
  'Hi there, what can I do for you?',
  'Hi there, how can I help?'],
 'items': ['We sell coffee and tea', 'We have coffee and tea'],
 'payments': ['We accept VISA, Mastercard and Paypal',
  'We accept most major credit cards, and Paypal'],
 'thanks': ['Happy to help!', 'Any time!', 'My pleasure']}

In [ ]:
data=pd.DataFrame({"inputs":inputs,"tags":tags})

In [ ]:
data.head(8),data.shape

(             inputs      tags
 0                Hi  greeting
 1               Hey  greeting
 2       How are you  greeting
 3  Is anyone there?  greeting
 4             Hello  greeting
 5          Good day  greeting
 6               Bye   goodbye
 7     See you later   goodbye, (26, 2))

In [ ]:
import string

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
data['inputs']=data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])

data['inputs']=data['inputs'].apply(lambda wrd:''.join(wrd))
data.shape



(26, 2)

In [ ]:
data.head(10)

,inputs,tags
0,hi,greeting
1,hey,greeting
2,how are you,greeting
3,is anyone there,greeting
4,hello,greeting
5,good day,greeting
6,bye,goodbye
7,see you later,goodbye
8,goodbye,goodbye
9,thanks,thanks


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
print(tokenizer.document_count)
print("The word index",tokenizer.word_index)

train=tokenizer.texts_to_sequences(data['inputs'])

print(train)


from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train=pad_sequences(train)


from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()


y_train=le.fit_transform(data['tags'])


26
The word index {'you': 1, 'do': 2, 'how': 3, 'are': 4, 'a': 5, 'take': 6, 'there': 7, 'thanks': 8, 'items': 9, 'what': 10, 'i': 11, 'long': 12, 'does': 13, 'delivery': 14, 'tell': 15, 'me': 16, 'joke': 17, 'hi': 18, 'hey': 19, 'is': 20, 'anyone': 21, 'hello': 22, 'good': 23, 'day': 24, 'bye': 25, 'see': 26, 'later': 27, 'goodbye': 28, 'thank': 29, 'thats': 30, 'helpful': 31, 'lot': 32, 'which': 33, 'have': 34, 'kinds': 35, 'of': 36, 'sell': 37, 'credit': 38, 'cards': 39, 'accept': 40, 'mastercard': 41, 'can': 42, 'pay': 43, 'with': 44, 'paypal': 45, 'cash': 46, 'only': 47, 'shipping': 48, 'when': 49, 'get': 50, 'my': 51, 'something': 52, 'funny': 53, 'know': 54}
[[18], [19], [3, 4, 1], [20, 21, 7], [22], [23, 24], [25], [26, 1, 27], [28], [8], [29, 1], [30, 31], [8, 5, 32], [33, 9, 2, 1, 34], [10, 35, 36, 9, 4, 7], [10, 2, 1, 37], [2, 1, 6, 38, 39], [2, 1, 40, 41], [42, 11, 43, 44, 45], [4, 1, 46, 47], [3, 12, 13, 14, 6], [3, 12, 13, 48, 6], [49, 2, 11, 50, 51, 14], [15, 16, 5, 17],

In [ ]:
y_train

array([3, 3, 3, 3, 3, 3, 2, 2, 2, 6, 6, 6, 6, 4, 4, 4, 5, 5, 5, 5, 0, 0,
       0, 1, 1, 1])

In [ ]:
x_train

array([[ 0,  0,  0,  0,  0, 18],
       [ 0,  0,  0,  0,  0, 19],
       [ 0,  0,  0,  3,  4,  1],
       [ 0,  0,  0, 20, 21,  7],
       [ 0,  0,  0,  0,  0, 22],
       [ 0,  0,  0,  0, 23, 24],
       [ 0,  0,  0,  0,  0, 25],
       [ 0,  0,  0, 26,  1, 27],
       [ 0,  0,  0,  0,  0, 28],
       [ 0,  0,  0,  0,  0,  8],
       [ 0,  0,  0,  0, 29,  1],
       [ 0,  0,  0,  0, 30, 31],
       [ 0,  0,  0,  8,  5, 32],
       [ 0, 33,  9,  2,  1, 34],
       [10, 35, 36,  9,  4,  7],
       [ 0,  0, 10,  2,  1, 37],
       [ 0,  2,  1,  6, 38, 39],
       [ 0,  0,  2,  1, 40, 41],
       [ 0, 42, 11, 43, 44, 45],
       [ 0,  0,  4,  1, 46, 47],
       [ 0,  3, 12, 13, 14,  6],
       [ 0,  3, 12, 13, 48,  6],
       [49,  2, 11, 50, 51, 14],
       [ 0,  0, 15, 16,  5, 17],
       [ 0,  0, 15, 16, 52, 53],
       [ 0,  2,  1, 54,  5, 17]], dtype=int32)

In [ ]:
input_shape=x_train.shape[1]
input_shape

6

In [ ]:
vocab=len(tokenizer.word_index)
print("num of unique words", vocab)
output_len=le.classes_.shape[0]

print("output length", output_len)


num of unique words 54
output length 7


In [ ]:
i=Input(shape=(input_shape,))
x=Embedding(vocab+1,10)(i)
x=LSTM(10,return_sequences=True)(x)
x=Flatten()(x)
x=Dense(output_len,activation='softmax')(x)
model=Model(i,x)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
train=model.fit(x_train,y_train,epochs=300)

Epoch 1/300
1/1 [==============================] - 1s 1s/step - loss: 1.9454 - accuracy: 0.1538
Epoch 2/300
1/1 [==============================] - 0s 10ms/step - loss: 1.9439 - accuracy: 0.1538
Epoch 3/300
1/1 [==============================] - 0s 9ms/step - loss: 1.9424 - accuracy: 0.1538
Epoch 4/300
1/1 [==============================] - 0s 9ms/step - loss: 1.9410 - accuracy: 0.1538
Epoch 5/300
1/1 [==============================] - 0s 9ms/step - loss: 1.9395 - accuracy: 0.1538
Epoch 6/300
1/1 [==============================] - 0s 7ms/step - loss: 1.9380 - accuracy: 0.1538
Epoch 7/300
1/1 [==============================] - 0s 8ms/step - loss: 1.9365 - accuracy: 0.1923
Epoch 8/300
1/1 [==============================] - 0s 7ms/step - loss: 1.9349 - accuracy: 0.1923
Epoch 9/300
1/1 [==============================] - 0s 8ms/step - loss: 1.9333 - accuracy: 0.3077
Epoch 10/300
1/1 [==============================] - 0s 9ms/step - loss: 1.9317 - accuracy: 0.3077
Epoch 11/300
1/1 [===========

In [ ]:
model.save('chatmodel.h5', train)


In [ ]:
import random

while True:
  texts=[]
  prediction_input=input("Enti : ")

  prediction_input=[letters.lower() for letters in prediction_input if letters not in string.punctuation]

  prediction_input=''.join(prediction_input)
  texts.append(prediction_input)
  prediction_input=tokenizer.texts_to_sequences(texts)
  prediction_input=np.array(prediction_input).reshape(-1)
  prediction_input=pad_sequences([prediction_input],input_shape)
  
  output=model.predict(prediction_input)
  output=output.argmax()
  response_tag=le.inverse_transform([output])[0]
  print("Helmi's CHATBOT : ",random.choice(responses[response_tag]))
  if response_tag=='goodbye':
    break


Enti : hi
Helmi's CHATBOT :  Hi there, what can I do for you?
Enti : what do you sell
Helmi's CHATBOT :  We have coffee and tea
Enti : alright bye
Helmi's CHATBOT :  See you later, thanks for visiting
